# Rapport de laboratoire 5
Hugo Fournier, 2011296

## Question 1
Implémentation de quad_penalty

In [10]:
using Krylov, LinearAlgebra, Logging, NLPModels, ADNLPModels, Printf, SolverCore, Test

In [11]:
using PDENLPModels, Gridap

In [12]:
function dsol(A, b, ϵ; solver :: Function = lsqr)
    (d, stats) = solver(A, b, atol = ϵ)
    return d
end

dsol (generic function with 1 method)

In [13]:
function quad_penalty_adnlp(nlp :: ADNLPModel, ρ :: Real)
    # TODO
    f_quad(x) = nlp.f(x) + (ρ / 2) * nlp.c(x)' * nlp.c(x)
    # f_quad(x) = nlp.f(x) + (ρ / 2) * (norm(nlp.c(x))^2)
    x0_quad = nlp.meta.x0
    nlp_quad = ADNLPModel(f_quad, x0_quad)
    return nlp_quad
end

quad_penalty_adnlp (generic function with 1 method)

In [14]:
function KKT_eq_constraint(nlp :: AbstractNLPModel, x, λ)
    # TODO
    KKT = grad(nlp, x) + jac(nlp, x)' * λ
    ϵ = 1e-3
    if norm(KKT) <= ϵ && norm(nlp.c(x)) <= ϵ
       return true
    else
       return false
    end
 end

KKT_eq_constraint (generic function with 1 method)

In [ ]:
# using NLPModelsIpopt

In [15]:
function quad_penalty(nlp      :: AbstractNLPModel, #Note que ça ne marche pas pour tous les NLPModel! 
    x        :: AbstractVector; 
    ϵ        :: AbstractFloat = 1e-3,
    η        :: AbstractFloat = 1e6, 
    σ        :: AbstractFloat = 2.0,
    max_eval :: Int = 1_000, 
    max_time :: AbstractFloat = 60.,
    max_iter :: Int = typemax(Int64)
    )
##### Initialiser cx et gx au point x;
# cx = nlp.c(x)       # Initialiser la violation des contraintes
cx = nlp.c(x)
gx = grad(nlp, x)   # Initialiser le gradient
######################################################
normcx = normcx_old = norm(cx)

ρ = 1.

iter = 0    

el_time = 0.0
tired   = neval_cons(nlp) > max_eval || el_time > max_time
status  = :unknown

start_time = time()
too_small  = false
normdual   = norm(gx) #exceptionnellement on ne va pas vérifier toute l'optimalité au début.
optimal    = max(normcx, normdual) ≤ ϵ

nlp_quad   = quad_penalty_adnlp(nlp, ρ)

@info log_header([:iter, :nf, :primal, :status, :nd, :Δ],
[Int, Int, Float64, String, Float64, Float64],
hdr_override=Dict(:nf => "#F", :primal => "‖F(x)‖", :nd => "‖d‖"))

while !(optimal || tired || too_small)

#Appeler Ipopt pour résoudre le problème pénalisé en partant du point x0 = x.
#utiliser l'option print_level = 0 pour enlever les affichages d'ipopt.
stats = ipopt(nlp_quad, x0 = x, print_level = 0)
################################################

if stats.status == :first_order
###### Mettre à jour cx avec la solution renvoyé par Ipopt
x = stats.solution
cx = nlp.c(x)
##########################################################
normcx_old = normcx
normcx = norm(cx)
end

if normcx_old > 0.95 * normcx
ρ *= σ
end

@info log_row(Any[iter, neval_cons(nlp), normcx, stats.status])

nlp_quad   = quad_penalty_adnlp(nlp, ρ)

el_time      = time() - start_time
iter   += 1
many_evals   = neval_cons(nlp) > max_eval
iter_limit   = iter > max_iter
tired        = many_evals || el_time > max_time || iter_limit || ρ ≥ η
##### Utiliser la réalisabilité dual renvoyé par Ipopt pour `normdual`
normdual     = stats.dual_feas
###################################################################
optimal      = max(normcx, normdual) ≤ ϵ
end

status = if optimal 
:first_order
elseif tired
if neval_cons(nlp) > max_eval
:max_eval
elseif el_time > max_time
:max_time
elseif iter > max_iter
:max_iter
else
:unknown_tired
end
elseif too_small
:stalled
else
:unknown
end

return GenericExecutionStats(nlp, status = status, solution = x,
               objective = obj(nlp, x),
               primal_feas = normcx,
               dual_feas = normdual,
               iter = iter, 
               elapsed_time = el_time,
               solver_specific = Dict(:penalty => ρ))
end

quad_penalty (generic function with 1 method)

Tests de quad_penalty sur un problème avec contrainte d'égalité de la collection OptimizationProblems.ADNLPProblems

In [21]:
using ADNLPModels

In [22]:
using OptimizationProblems, OptimizationProblems.ADNLPProblems

In [23]:
# problem = BNST2(3)
# nlp_problem = MathOptNLPModel(problem)

# quad_penalty(nlp_problem, nlp_problem.meta.x0)

problems = OptimizationProblems.meta[!, :name]

# nlp = OptimizationProblems.ADNLPProblems.zangwil3()
nlp = BOX2()

20-element Vector{String}:
 "AMPGO02"
 "AMPGO03"
 "AMPGO04"
 "AMPGO05"
 "AMPGO06"
 "AMPGO07"
 "AMPGO08"
 "AMPGO09"
 "AMPGO10"
 "AMPGO11"
 "AMPGO12"
 "AMPGO13"
 "AMPGO14"
 "AMPGO15"
 "AMPGO18"
 "AMPGO20"
 "AMPGO21"
 "AMPGO22"
 "BOX2"
 "BOX3"

MethodError: MethodError: no method matching ADNLPModel(::OptimizationProblems.ADNLPProblems.var"#f#49"{Int64}, ::Vector{Float64}, ::Vector{Int64}, ::Vector{Int64}, ::Vector{Float64}, ::Vector{Float64}, ::Vector{Float64}; name="BOX2")
Closest candidates are:
  ADNLPModel(::Any, ::S, !Matched::S, !Matched::S, ::Any, ::S, ::S; y0, name, lin, adbackend) where S at C:\Users\Goglu\.julia\packages\ADNLPModels\9yjtG\src\nlp.jl:121
  ADNLPModel(::Any, ::S, ::Any, !Matched::S, ::S; y0, name, lin, adbackend) where S at C:\Users\Goglu\.julia\packages\ADNLPModels\9yjtG\src\nlp.jl:83
  ADNLPModel(!Matched::NLPModelMeta{T, S}, !Matched::Counters, !Matched::ADNLPModels.ADBackend, ::Any, ::Any) where {T, S} at C:\Users\Goglu\.julia\packages\ADNLPModels\9yjtG\src\nlp.jl:4 got unsupported keyword argument "name"
  ...